In [23]:
import pandas as pd

In [24]:
df = pd.read_csv("waze_original.csv", sep=";")

In [25]:
df = df[df.Country == "LO"]
df = df[df.City == "Košice"]

In [26]:
import time
import calendar

In [27]:
epoch_times = []
month_dict = {"jan": "1", "feb": "2", "mar": "3", "apr": "4", "may": "5", "jun": "6", "jul": "7", "aug": "8", "sep": "9", "oct": "10", "nov": "11", "dec": "12"}
for index, row in df.iterrows():
    month, day = 1, 1
    try:
        month, day = row["Date"].split(".")
    except:
        month, day = row["Date"].split(" ")
        
    utc_time = time.strptime(".".join([str(row["Year"]), month_dict[month.lower()], day]), "%Y.%m.%d")
    epoch_time = calendar.timegm(utc_time)
    epoch_times.append(epoch_time)

In [28]:
df["Timestamp"] = epoch_times

In [29]:
df = df.drop('Date', axis=1).drop('Year', axis=1).drop('Country', axis=1).drop('City', axis=1).drop('Type', axis=1)

In [30]:
df = df[df.Subtype != "HAZARD_WEATHER"]

In [31]:
import json
alerts_dict = {}
with open("alerts.json", "r") as alerts_file:
    alerts_dict = json.load(alerts_file)

In [32]:
types = []
lats = [] #48
lngs = [] #21
for index, row in df.iterrows():
    _type = None
    try:
        _type = alerts_dict[row.Subtype]
    except:
        _type = "DELETE"
        
    lng, lat = row.Location.replace("Point(", "")[:-1].split(" ")[::-1]
    
    types.append(_type)
    lats.append(lng)
    lngs.append(lat)

df["Type"] = types
df["Lat"] = lats
df["Lng"] = lngs

df = df.drop('Location', axis=1).drop('Subtype', axis=1)
df = df[df.Type != "DELETE"]

In [35]:
df.to_csv("waze-all_clean.csv", index=False)

In [70]:
df.sort_values(by=['Timestamp'], ascending=False)

most_recent_timestamp = df.iloc[[0]].Timestamp

df = df[most_recent_timestamp[1] - df.Timestamp <= 2678400] #31 days

In [72]:
df.to_csv("waze-history-month.csv", index=False)

In [74]:
most_recent_timestamp = df.iloc[[0]].Timestamp

df = df[most_recent_timestamp[1] - df.Timestamp <= 86400] #1 day

In [76]:
df.to_csv("waze-history-day.csv", index=False)